In [1]:
# Import core libraries for data manipulation and visualization
from matplotlib.pyplot import subplots  # For creating figure and axis objects for plotting
import numpy as np  # Numerical computing library for array operations
import pandas as pd  # Data manipulation library for DataFrames
from ISLP.models import ModelSpec as MS  # Model specification tool for creating design matrices

# Import survival analysis tools from the lifelines package
from lifelines import (
    KaplanMeierFitter,  # Fits Kaplan-Meier survival curves (non-parametric estimator)
    CoxPHFitter  # Fits Cox Proportional Hazards models (semi-parametric regression)
)
from lifelines.statistics import (
    logrank_test,  # Tests equality of survival curves between 2 groups
    multivariate_logrank_test  # Tests equality of survival curves among 3+ groups
)

print("All libraries imported successfully!")

All libraries imported successfully!


In [2]:
# Set random seed for reproducibility (following ISLP lab convention)
rng = np.random.default_rng(2025)

# Simulate 500 employees hired over a 5-year period
n_employees = 500

# Department assignment (5 departments with realistic proportions)
# Engineering is largest, HR smallest (typical for tech companies)
departments = rng.choice(
    ['Engineering', 'Sales', 'Marketing', 'HR', 'Operations'], 
    size=n_employees, 
    p=[0.40, 0.25, 0.18, 0.08, 0.09]
)

# Years of prior work experience at hire (0-20 years)
# Use exponential distribution to create right-skewed realistic experience levels
experience = rng.exponential(scale=4.5, size=n_employees)
experience = np.clip(experience, 0, 20)

# Performance rating (1.0-5.0 scale, where higher is better)
# Centered at 3.5 with standard deviation of 0.7 to create realistic distribution
performance = rng.normal(loc=3.5, scale=0.7, size=n_employees)
performance = np.clip(performance, 1.0, 5.0)

# Starting salary in thousands (correlated with experience and department)
# Different departments have different base salary structures
dept_salary_base = {
    'Engineering': 95,  # Highest base (competitive tech market)
    'Sales': 72,        # Lower base but typically has commission
    'Marketing': 78,    # Moderate base
    'HR': 68,           # Lower base
    'Operations': 85    # Good base (technical operations)
}
base_salary = np.array([dept_salary_base[d] for d in departments])
# Add experience premium (~$2.5k per year) plus random variation
salary = base_salary + 2.5 * experience + rng.normal(0, 8, n_employees)
salary = np.clip(salary, 45, 160)

# Remote work assignment (binary: 1 = remote, 0 = on-site)
# Engineering has highest remote proportion, Operations lowest
remote_prob = np.where(
    departments == 'Engineering', 0.60,  # Engineering: 60% remote
    np.where(departments == 'Operations', 0.15,  # Operations: 15% remote
             np.where(departments == 'Sales', 0.40,  # Sales: 40% remote
                     np.where(departments == 'Marketing', 0.45, 0.35)))  # Marketing: 45%, HR: 35%
)
remote_work = rng.binomial(1, remote_prob)

print("Covariates generated successfully!")
print(f"Sample size: {n_employees} employees")
print(f"Remote work proportion: {remote_work.mean():.1%}")

Covariates generated successfully!
Sample size: 500 employees
Remote work proportion: 45.2%


In [3]:
# Simulate employment duration (time until departure) using Cox model structure
# Different departments have different baseline retention patterns
dept_hazard_effects = {
    'Engineering': -0.40,   # Best retention (negative = lower hazard)
    'Sales': 0.45,          # Worst retention (positive = higher hazard)
    'Marketing': 0.18,      # Moderate-poor retention
    'HR': 0.08,             # Moderate retention
    'Operations': -0.25     # Good retention
}
dept_effect = np.array([dept_hazard_effects[d] for d in departments])

# Calculate hazard rate (risk of leaving) based on multiple factors
# Cox model structure: λ(t) = λ₀(t) × exp(β₁X₁ + β₂X₂ + ...)
# Protective factors (negative coefficients): experience, performance, salary, remote work
log_hazard = (
    dept_effect +                          # Department effect
    -0.10 * experience +                   # Each year experience reduces hazard by 9.5% (exp(-0.10)≈0.905)
    -0.40 * (performance - 3.5) +          # Higher performance is strongly protective
    -0.018 * (salary - 85) +               # Higher salary reduces departure risk
    -0.35 * remote_work +                  # Remote work reduces hazard by 30% (exp(-0.35)≈0.70)
    -1.5                                   # Baseline log-hazard
)
lambda_hazard = np.exp(log_hazard)  # Convert to hazard rate

# Generate time to departure from exponential distribution (months)
# Exponential distribution is memoryless and commonly used in survival simulation
time_to_departure = rng.exponential(scale=1/lambda_hazard, size=n_employees)

# Add administrative censoring (employees still at company at study end)
# Study duration varies by employee (12-60 months observation window)
# This creates realistic Type I censoring
study_duration = rng.uniform(low=12, high=60, size=n_employees)

# Observed time is minimum of departure time and censoring time
observed_time = np.minimum(time_to_departure, study_duration)
# Event indicator: 1 = departed (event observed), 0 = still employed (censored)
event_indicator = (time_to_departure <= study_duration).astype(int)

print(f"Survival times generated!")
print(f"Events (departures): {event_indicator.sum()}")
print(f"Censored (still employed): {(1 - event_indicator).sum()}")
print(f"Censoring rate: {(1 - event_indicator).mean():.1%}")

Survival times generated!
Events (departures): 433
Censored (still employed): 67
Censoring rate: 13.4%


In [4]:
# Create final DataFrame with all variables
retention_data = pd.DataFrame({
    'time': observed_time,                    # Employment duration in months
    'event': event_indicator,                 # 1 = left company, 0 = still employed
    'department': pd.Categorical(departments), # Department (categorical)
    'experience': experience,                 # Years of prior experience (continuous)
    'performance': performance,               # Performance rating 1-5 (continuous)
    'salary': salary,                         # Starting salary in thousands (continuous)
    'remote_work': remote_work                # 1 = remote, 0 = on-site (binary)
})

print("=" * 70)
print("EMPLOYEE RETENTION DATASET CREATED")
print("=" * 70)
print(f"Dataset dimensions: {retention_data.shape[0]} employees × {retention_data.shape[1]} variables")
print(f"\nFirst 10 observations:")
print(retention_data.head(10))

EMPLOYEE RETENTION DATASET CREATED
Dataset dimensions: 500 employees × 7 variables

First 10 observations:
        time  event   department  experience  performance      salary  \
0   4.137837      1   Operations    0.841920     2.655509   92.316205   
1   3.282764      1  Engineering    0.946957     3.380019   92.603354   
2  18.432079      1    Marketing    9.409169     2.735693   98.144888   
3   4.663205      1           HR    5.583752     2.030922   76.167897   
4   2.323511      1   Operations    0.686558     3.539874   79.926189   
5  14.018088      1  Engineering    2.070775     3.140689  104.693788   
6  39.645606      0  Engineering   15.157682     4.367160  138.590670   
7   4.476022      1   Operations    3.327090     3.320088   81.560671   
8   4.057824      1    Marketing    2.883564     3.737368   96.047723   
9  16.240357      0  Engineering   15.727998     3.492641  123.907938   

   remote_work  
0            0  
1            1  
2            1  
3            1  
4   